ASAP CRN Metadata validation - wave 1

# Team Lee. ASAP CRN Metadata validation - wave 1



28 Nov 2023
Andy Henrie




In [95]:
import pandas as pd

from pathlib import Path

from utils.io import read_meta_table, get_dtypes_dict, ReportCollector
from utils.qcutils import prep_table, validate_table, validate_file, read_ASAP_CDE, reorder_table_to_CDE



%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Clean V1 Table
write clean metadata tables according to CDE v1

### Team Lee

In [96]:
## convert 
data_path = Path.home() / ("Projects/ASAP/team-lee")
metadata_path = data_path / "metadata/ogmetadata"

SUBJECT = pd.read_csv(f"{metadata_path}/SUBJECT.tsv", delimiter="\t")
SAMPLE = pd.read_csv(f"{metadata_path}/SAMPLE.tsv",delimiter="\t")

CLINPATH = pd.read_csv(f"{metadata_path}/CLINPATH.csv",delimiter=",")
STUDY = pd.read_csv(f"{metadata_path}/STUDY.tsv",delimiter="\t")
PROTOCOL = pd.read_csv(f"{metadata_path}/PROTOCOL.tsv",delimiter="\t")

metadata_version = "v1"

CDE_df,dtypes_dict = read_ASAP_CDE(metadata_version=metadata_version)
METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"

read url


In [97]:
STUDY = pd.read_csv(metadata_path / "STUDY.tsv",delimiter="\t")
STUDY.to_csv(data_path / "STUDY_.csv")
STUDY = pd.read_csv(data_path / "STUDY_.csv")
STUDY.head()


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Team-Lee-Bras-Lab-Info,Field,Description,Data type,Validation,Note,Required/Optional
0,Is senescence a component of human PD and does...,project_name,Project Name/Title,String,NaN,Unique and clear title.,Required,NaN,NaN,NaN
1,Human snRNA-seq PD Senesence Jose Bras Team Lee,project_dataset,Dataset name,String,NaN,A Dataset name is required for each submission...,Required,NaN,NaN,NaN
2,Characterize the neuropathological progression...,project_description,Brief description of the goals and objectives ...,String,NaN,NaN,Required,NaN,NaN,NaN
3,TEAM-LEE,ASAP_team_name,"ASAP Team e.g. ""Scherzer""",Enum,"[""TEAM-LEE"",""TEAM-HAFLER"",""TEAM-HARDY"",....]",NaN,Required,NaN,NaN,NaN
4,Bras,ASAP_lab_name,"ASAP Lab under the above team e.g. ""Dong""",String,NaN,NaN,Required,NaN,NaN,NaN


In [98]:

# fix STUDY formatting
tmp = pd.DataFrame()
tmp = STUDY[["Unnamed: 1","Unnamed: 0"]].transpose().reset_index().drop(columns=["index"])
tmp.columns = tmp.iloc[0]
STUDY = tmp.drop([0])
# STUDY[["Unnamed: 1"]].transpose().reset_index().drop(columns=["index"]), tmp
STUDY.head()

,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,submitter_id,submitter_name,submittor_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date
1,Is senescence a component of human PD and does...,Human snRNA-seq PD Senesence Jose Bras Team Lee,Characterize the neuropathological progression...,TEAM-LEE,Bras,"Jose, Bras",jose.bras@vai.org,"Lee, L, Marshall ; Kimberly, E, Paquette ; Kai...",Kaitlyn E Westra,kaitlyn.westra@vai.org,...,NaN,NaN,NaN,75,hippocampus; middle frontal gyrus; substantia ...,human PD and control postmortem brains,NaN,NaN,unsure,NaN


In [99]:
metadata_version_date = "v1, SEPT2023"
STUDY["metadata_version_date"] = METADATA_VERSION_DATE
# Testing the function with STUDY.csv and CDE.csv
STUDY.replace("Nan", "", inplace=True)

# Need to rename submitter_id to contributor_names
STUDY = STUDY.rename(columns={"submitter_id": "contributor_names"})

report = ReportCollector(destination="")

validate_table(STUDY, "STUDY", CDE_df, report)

report.print_log()


🚨⚠️❗ **Missing Required Fields in STUDY: submitter_email**
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in STUDY. 🥳



Collect additional metadata from covar.csv .. i.e. batch

In [100]:

metadata_path = Path.home() / ("Projects/ASAP/team-lee/metadata")
HIP_covar = pd.read_csv(f"{metadata_path}/HIP/covar.csv")
HIP_cases = pd.read_csv(f"{metadata_path}/HIP/PD_ASAP_Sample_batch_information_banner_cases.csv").dropna(axis=0,how='all')
HIP_control = pd.read_csv(f"{metadata_path}/HIP/PD_ASAP_Sample_batch_information_banner_controls.csv")

MFG_covar = pd.read_csv(f"{metadata_path}/MFG/covar.csv") # includes 'PMI' ?
MFG_cases = pd.read_csv(f"{metadata_path}/MFG/PD_ASAP_Sample_batch_information_banner_cases.csv").dropna(axis=0,how='all')
MFG_control = pd.read_csv(f"{metadata_path}/MFG/PD_ASAP_Sample_batch_information_banner_controls.csv")


SN_covar = pd.read_csv(f"{metadata_path}/SN/covar.csv")
SN_cases = pd.read_csv(f"{metadata_path}/SN/PD_ASAP_Sample_batch_information_banner_cases.csv").dropna(axis=0,how='all')
SN_control = pd.read_csv(f"{metadata_path}/SN/PD_ASAP_Sample_batch_information_banner_controls.csv")

In [101]:
# Hippocampus samples
# HIP_cases["GROUPcv"]="PD"
# HIP_control["GROUPcv"]="HC"

HIP_meta = pd.concat([HIP_cases, HIP_control], axis=0, ignore_index=True)
HIP_meta["GROUPcv"]= HIP_meta["PD"].apply(lambda x: "PD" if (x=="yes") else "HC")


In [102]:


HIP_meta['MERGE_ID'] = "HIP_" + HIP_meta['GROUPcv'] +"_" + HIP_meta['CaseID'].str.replace('-','')
HIP_covar['MERGE_ID'] = HIP_covar['COUNT_ID']
# the fastqs follow COUNT_ID insteald of SEQ_ID naming convention
HIP_covar['SEQ_ID'] = HIP_covar['COUNT_ID']



In [103]:
# there's a bug in the meta table... skip for now
HIP_TABLE = pd.merge(HIP_covar, HIP_meta, on='MERGE_ID', how='outer')

# HIP_TABLE = HIP_covar
HIP_TABLE['subdir']="HIP"


In [104]:
test = HIP_TABLE[["MERGE_ID","SEQ_ID","GROUPcv","subdir",'PD']]

In [105]:
### medial frontal gyrus samples
MFG_meta = pd.concat([MFG_cases, MFG_control], axis=0, ignore_index=True)
MFG_meta["GROUPcv"]= MFG_meta["PD"].apply(lambda x: "PD" if (x=="yes") else "HC")

# make a MERGE_ID column because the formatting is inconsistent
MFG_meta['MERGE_ID'] = "MFG_" + MFG_meta['GROUPcv'] +"_" + MFG_meta['CaseID'].str.replace('-','')
MFG_covar['MERGE_ID'] = MFG_covar['SAMPLE']
# the fastqs are in SEQ_ID 

# there's a bug in the meta table... skip for now
MFG_TABLE = pd.merge(MFG_covar, MFG_meta, on='MERGE_ID', how='inner')
MFG_TABLE['subdir']="MFG"



# Substantia Nigra
SN_meta = pd.concat([SN_cases, SN_control], axis=0, ignore_index=True)
SN_meta["GROUPcv"] = SN_meta["PD"].apply(lambda x: "PD" if (x=="yes") else "HC")

SN_meta['MERGE_ID'] = "SN_" + MFG_meta['GROUPcv'] +"_" + MFG_meta['CaseID'].str.replace('-','')
SN_covar['MERGE_ID'] = SN_covar['SAMPLE']

# there's a bug in the meta table... skip for now
SN_TABLE = pd.merge(SN_covar, SN_meta, on='MERGE_ID', how='outer')
SN_TABLE['subdir']="SN"


### concatenate SN, MSG, and HIP tables into one 'all_samples' table
all_samples = pd.concat([HIP_TABLE, MFG_TABLE, SN_TABLE], axis=0, ignore_index=True)


In [106]:

SAMPLE_ALL = SAMPLE.merge(all_samples, left_on='sample_id', right_on='MERGE_ID', how='left')
SAMPLE_ALL.to_csv("alternate_metadata.csv")

In [107]:
SAMPLE_og = SAMPLE.copy()
SAMPLE['batch'] = SAMPLE_ALL['BATCH']

In [108]:
# Extract the fields with DataType as "Enum" or "String" for the "sample" table from CDE.csv
SAMPLE = prep_table(SAMPLE, CDE_df)
# for field in string_enum_fields:
#     if field in SAMPLE.columns:
#         SAMPLE[field] = SAMPLE[field].astype(str)


In [109]:
# Testing the function with PROTOCOL.csv and CDE.csv

report = ReportCollector(destination="")
validate_table(PROTOCOL, "PROTOCOL", CDE_df, report)
report.print_log()

All required fields are present in *PROTOCOL* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in PROTOCOL. 🥳



In [110]:
# Extract the fields with DataType as "Enum" or "String" for the "sample" table from CDE.csv
SUBJECT.replace("Nan", "", inplace=True)


SUBJECT = prep_table(SUBJECT, CDE_df)


report = ReportCollector(destination="logs")
validate_table(SUBJECT, "SUBJECT", CDE_df, report)
report.print_log()

All required fields are present in *SUBJECT* table.
No empty entries (Nan) found in _Required_ fields.
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- primary_diagnosis_text: 23/25 empty rows
## Enum fields have valid values in SUBJECT. 🥳



In [111]:
SUBJECT.head()

,subject_id,source_subject_id,biobank_name,organism,sex,age_at_collection,race,ethnicity,duration_pmi,primary_diagnosis,primary_diagnosis_text
0,HC_1225,12-25,Banner Sun Health Research Institute,Human,Male,80,White,Not Reported,3.50,No PD nor other neurological disorder,NaN
1,HC_0602,06-02,Banner Sun Health Research Institute,Human,Male,84,White,Not Reported,2.66,Other neurological disorder,Mild Cognitive Impairment
2,PD_0009,00-09,Banner Sun Health Research Institute,Human,Male,64,White,Not Reported,4.00,Idiopathic PD,NaN
3,PD_1921,19-21,Banner Sun Health Research Institute,Human,Male,82,White,Not Reported,3.93,Idiopathic PD,NaN
4,PD_2058,20-58,Banner Sun Health Research Institute,Human,Male,87,White,Not Reported,3.17,Idiopathic PD,NaN


In [112]:
SAMPLE.replace("Nan", "", inplace=True)

SAMPLE = prep_table(SAMPLE, CDE_df)


In [113]:
SAMPLE['file_type'] = SAMPLE['file_type'].replace({"Fastq":"fastq"})


In [114]:
# fix file_name and file_MD5 which need to be exploded (do this last for simplicity. i.e. to keep one sample per row rather than one file per row)

# Step 1: Split the values in the columns based on commas
SAMPLE['file_name'] = SAMPLE['file_name'].str.split(',')
SAMPLE['file_MD5(R1,R2)'] = SAMPLE['file_MD5(R1,R2)'].str.split(',')

# Step 2: Explode both 'file_name' and 'file_MD5(R1,R2)' columns together
SAMPLE = SAMPLE.explode(['file_name', 'file_MD5(R1,R2)'])

# Step 3: Rename the "file_MD5(R1,R2)" column to "file_MD5"
SAMPLE = SAMPLE.rename(columns={"file_MD5(R1,R2)": "file_MD5"})



In [115]:


report = ReportCollector(destination="")
validate_table(SAMPLE, "SAMPLE", CDE_df, report)
report.print_log()


All required fields are present in *SAMPLE* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- source_RIN: 150/150 empty rows

	- RIN: 150/150 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- pm_PH: 150/150 empty rows
## Enum fields have valid values in SAMPLE. 🥳



In [116]:
# fix subject id in SAMPLE

samp_id_mapper = dict(zip(SUBJECT['source_subject_id'], SUBJECT['subject_id']))
SAMPLE['subject_id'] = SAMPLE['subject_id'].map(samp_id_mapper)

In [117]:
SAMPLE.head()

,sample_id,source_sample_id,subject_id,replicate,replicate_count,repeated_sample,tissue,brain_region,source_RIN,RIN,...,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV2000,pm_PH,donor_id,batch
0,MFG_HC_1225,12-25,HC_1225,rep1,1,0,Brain,Middle_Frontal_Gyrus,NaN,NaN,...,Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NaN,NaN,,BATCH_4
0,MFG_HC_1225,12-25,HC_1225,rep1,1,0,Brain,Middle_Frontal_Gyrus,NaN,NaN,...,Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NaN,NaN,,BATCH_4
1,MFG_HC_0602,06-02,HC_0602,rep1,1,0,Brain,Middle_Frontal_Gyrus,NaN,NaN,...,Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NaN,NaN,,BATCH_4
1,MFG_HC_0602,06-02,HC_0602,rep1,1,0,Brain,Middle_Frontal_Gyrus,NaN,NaN,...,Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NaN,NaN,,BATCH_4
2,MFG_PD_0009,00-09,PD_0009,rep1,1,0,Brain,Middle_Frontal_Gyrus,NaN,NaN,...,Unknown,MONDO:0005180,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NaN,NaN,,BATCH_4


In [118]:

SAMPLE['file_name'].replace({'HIP_PD_0348_S0_L000_R2_001.fastq.gzgz':'HIP_PD_0348_S0_L000_R2_001.fastq.gz',
                              'HIP_HC_1939_S6_L001_R2_001.fastq.gzgz':  'HIP_HC_1939_S6_L001_R2_001.fastq.gz'},inplace=True)
SAMPLE.file_name.to_list()


['MFGHC1225_S9_L001_R1_001.fastq.gz',
 'MFGHC1225_S9_L001_R2_001.fastq.gz',
 'MFGHC0602_S2_L001_R1_001.fastq.gz',
 'MFGHC0602_S2_L001_R2_001.fastq.gz',
 'MFGPD0009_S3_L001_R1_001.fastq.gz',
 'MFGPD0009_S3_L001_R2_001.fastq.gz',
 'MFGPD1921_S9_L001_R1_001.fastq.gz',
 'MFGPD1921_S9_L001_R2_001.fastq.gz',
 'MFGPD2058_S5_L001_R1_001.fastq.gz',
 'MFGPD2058_S5_L001_R2_001.fastq.gz',
 'MFGPD1441_S9_L001_R1_001.fastq.gz',
 'MFGPD1441_S9_L001_R2_001.fastq.gz',
 'MFGPD1344_S7_L001_R1_001.fastq.gz',
 'MFGPD1344_S7_L001_R2_001.fastq.gz',
 'MFGHC1939_S15_L001_R1_001.fastq.gz',
 'MFGHC1939_S15_L001_R2_001.fastq.gz',
 'MFGHC1308_S8_L002_R1_001.fastq.gz',
 'MFGHC1308_S8_L002_R2_001.fastq.gz',
 'MFGHC1862_S5_L004_R1_001.fastq.gz',
 'MFGHC1862_S5_L004_R2_001.fastq.gz',
 'MFGHC1864_S1_L002_R1_001.fastq.gz',
 'MFGHC1864_S1_L002_R2_001.fastq.gz',
 'MFGHC2057_S7_L002_R1_001.fastq.gz',
 'MFGHC2057_S7_L002_R2_001.fastq.gz',
 'MFGHC2061_S7_L004_R1_001.fastq.gz',
 'MFGHC2061_S7_L004_R2_001.fastq.gz',
 'MFGHC206

In [119]:
CLINPATH = prep_table(CLINPATH, CDE_df)


CLINPATH.replace("Nan", "", inplace=True)

report = ReportCollector(destination="")
validate_table(CLINPATH, "CLINPATH", CDE_df, report)
report.print_log()


All required fields are present in *CLINPATH* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- age_at_onset: 75/75 empty rows

	- age_at_diagnosis: 75/75 empty rows

	- first_motor_symptom: 75/75 empty rows

	- path_year_death: 75/75 empty rows

	- brain_weight: 75/75 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- smoking_years: 75/75 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*region_level_2*_:  invalid values 💩'Hippocampus'
    - valid ➡️ 'Superior frontal gyrus', 'Middle frontal gyrus', 'Inferior frontal gyrus', 'Superior temporal gyrus', 'Middle temporal gyrus', 'Inferior temporal gyrus', 'Fusiform gyrus', 'Transentorhinal region', 'Entorinal region', 'Subiculum', 'CA1-CA4', 'Amygdala', 'Periamygdala cortex', 'Anterior cingulate gyrus', 'Posterior cingulate gyrus', 'Superior parietal lobule', 'Inferior parietal lobule', 'Parastriate cortex', 'Peristriate cortex', 'Striate cortex', 'Insular cortex', 'Caudate nucleus', 'Putamen', 'Globus pallidus', 

In [120]:
CLINPATH['region_level_2'].unique()

# change "Hippocampus" to "CA1-CA4"
CLINPATH['region_level_2'] = CLINPATH['region_level_2'].replace('Hippocampus', 'CA1-CA4')

# skip hx_melanoma and education level for now as there is not a "Unknown" or "Not Reported" option in the CDE

# leave te APOE_e4_status as is for now . multiple are coded as "2,3" 
# leave cognitive status as is, since there is no "Unknown" or "Not Reported" option in the CDE

# potential "path_braak_asyn" coding 
braak_map = {'L. Olfactory Bulb-Only':"1/2", 'Lla. Brainstem Predominant':"3",
       'Llb. Limbic Predominant':"3/4", 'LV. Neocortical':"5",
       'Lll. Brainstem/Limbic':"3/4", '0. No Lewy bodies':"0"}
# set to NaN for now since this is actualy path_mckeith coding

CLINPATH['path_braak_asyn'] = ""

mckeith_map = {'L. Olfactory Bulb-Only':"Olfactory bulb only", 'Lla. Brainstem Predominant':"Brainstem",
       'Llb. Limbic Predominant':"Limbic (transitional)", 'LV. Neocortical':"Neocortical",
       'Lll. Brainstem/Limbic':"Amygdala Predominant", '0. No Lewy bodies':"Absent"}


CLINPATH['path_mckeith'] = CLINPATH['path_mckeith'].replace(mckeith_map)

# leave path_nia_ri like this for now. not sure how to map "criteria not met" and "Not AD"

# leave amyloid_angiopathy_severity_scale like this for now. not sure how to map 'Cerebral amyloid angiopathy, temporal and occipital lobe','Cerebral amyloid angiopathy, frontal lobe']


In [121]:
report = ReportCollector(destination="")
validate_table(CLINPATH, "CLINPATH", CDE_df, report)
report.print_log()


All required fields are present in *CLINPATH* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- age_at_onset: 75/75 empty rows

	- age_at_diagnosis: 75/75 empty rows

	- first_motor_symptom: 75/75 empty rows

	- path_year_death: 75/75 empty rows

	- brain_weight: 75/75 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- smoking_years: 75/75 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*path_nia_ri*_:  invalid values 💩'Criteria not met', 'Not AD'
    - valid ➡️ 'Low', 'Intermediate', 'High', 'None'
- _*TDP43*_:  invalid values 💩'', 'Na'
    - valid ➡️ 'None in medial temporal lobe', 'Present in amygdala, only', 'Present in hippocampus, only', 'Present in amygdala and hippocampus, only', 'Present in medial temporal lobe and middle frontal gyrus (not FTLD pattern)', 'Unknown'
- _*amyloid_angiopathy_severity_scale*_:  invalid values 💩'', 'Cerebral amyloid angiopathy, temporal and occipital lobe', 'Cerebral amyloid angiopathy, frontal lobe'
    - valid ➡️ 'None', 

In [122]:

SAMPLE_ALL_CP = SAMPLE_ALL.merge(CLINPATH, on='sample_id', how='outer')


In [123]:
SAMPLE_ALL_CP.to_csv("./clean/team-Lee/auxiluary_metadata.csv")

In [124]:
data_path

PosixPath('/Users/ergonyc/Projects/ASAP/team-lee')

In [125]:
# fix the column order
STUDY = reorder_table_to_CDE(STUDY, "STUDY", CDE_df)
SAMPLE = reorder_table_to_CDE(SAMPLE, "SAMPLE", CDE_df)
PROTOCOL = reorder_table_to_CDE(PROTOCOL, "PROTOCOL", CDE_df)
SUBJECT = reorder_table_to_CDE(SUBJECT, "SUBJECT", CDE_df)     
CLINPATH = reorder_table_to_CDE(CLINPATH, "CLINPATH", CDE_df)


# write the clean metadata
STUDY.to_csv(data_path / "metadata/STUDY.csv")
PROTOCOL.to_csv(data_path / "metadata/PROTOCOL.csv")
CLINPATH.to_csv(data_path / "metadata/CLINPATH.csv")
SAMPLE.to_csv(data_path / "metadata/SAMPLE.csv")
SUBJECT.to_csv(data_path / "metadata/SUBJECT.csv")

# also writh them to clean...
# 
#  

export_root = Path.cwd() / "clean/team-Lee"
if not export_root.exists():
    export_root.mkdir(parents=True, exist_ok=True)


STUDY.to_csv( export_root / "STUDY.csv")
PROTOCOL.to_csv(export_root / "PROTOCOL.csv")
SAMPLE.to_csv(export_root / "SAMPLE.csv")
SUBJECT.to_csv(export_root / "SUBJECT.csv")
CLINPATH.to_csv(export_root / "CLINPATH.csv")


added empty column submitter_email to STUDY table
added empty column DV200 to SAMPLE table
added empty column dig_slide_avail to CLINPATH table
added empty column quant_path_avail to CLINPATH table


## Update the table to v2

In [126]:
SAMPLE.head()

,sample_id,source_sample_id,subject_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,source_RIN,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV200,pm_PH,donor_id
0,MFG_HC_1225,12-25,HC_1225,rep1,1,0,BATCH_4,Brain,Middle_Frontal_Gyrus,NaN,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,<NA>,NaN,
0,MFG_HC_1225,12-25,HC_1225,rep1,1,0,BATCH_4,Brain,Middle_Frontal_Gyrus,NaN,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,<NA>,NaN,
1,MFG_HC_0602,06-02,HC_0602,rep1,1,0,BATCH_4,Brain,Middle_Frontal_Gyrus,NaN,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,<NA>,NaN,
1,MFG_HC_0602,06-02,HC_0602,rep1,1,0,BATCH_4,Brain,Middle_Frontal_Gyrus,NaN,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,<NA>,NaN,
2,MFG_PD_0009,00-09,PD_0009,rep1,1,0,BATCH_4,Brain,Middle_Frontal_Gyrus,NaN,...,PATO:0000384 (male),Unknown,MONDO:0005180,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,<NA>,NaN,


In [127]:
from update_schema import update_tables_to_CDEv2

tables_path = Path.cwd() / "clean/team-Lee"


CDEv1 = pd.read_csv( Path.cwd() / "ASAP_CDE_v1.csv" )
CDEv2 = pd.read_csv( Path.cwd() / "ASAP_CDE_v2.csv" )


STUDYv2, PROTOCOLv2, SAMPLEv2, SUBJECTv2, CLINPATHv2, DATAv2 = update_tables_to_CDEv2(tables_path, CDEv1, CDEv2)


In [128]:

export_root = Path.cwd() / "clean/team-Lee/v2"
if not export_root.exists():
    export_root.mkdir(parents=True, exist_ok=True)


In [129]:

STUDYv2.to_csv( export_root / "STUDY.csv")
PROTOCOLv2.to_csv(export_root / "PROTOCOL.csv")
SAMPLEv2.to_csv(export_root / "SAMPLE.csv")
SUBJECTv2.to_csv(export_root / "SUBJECT.csv")
CLINPATHv2.to_csv(export_root / "CLINPATH.csv")
DATAv2.to_csv(export_root / "DATA.csv")


In [130]:
export_root

PosixPath('/Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Lee/v2')

Transfer cleaned metadata to raw buckets 




## Lee

In [131]:
# Lee
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/lee-credentials.json 


Activated service account credentials for: [raw-admin-lee@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [132]:

!gsutil -u dnastack-asap-parkinsons ls -al "gs://asap-raw-data-team-lee/metadata/v2"


      6243  2023-11-29T22:26:06Z  gs://asap-raw-data-team-lee/metadata/v2/CLINPATH.csv#1701296766077305  metageneration=1
     24218  2023-11-29T22:26:06Z  gs://asap-raw-data-team-lee/metadata/v2/DATA.csv#1701296766689376  metageneration=1
       968  2023-11-29T22:26:07Z  gs://asap-raw-data-team-lee/metadata/v2/PROTOCOL.csv#1701296767085192  metageneration=1
     23092  2023-11-29T22:26:06Z  gs://asap-raw-data-team-lee/metadata/v2/SAMPLE.csv#1701296766486332  metageneration=1
      1054  2023-11-29T22:26:06Z  gs://asap-raw-data-team-lee/metadata/v2/STUDY.csv#1701296766878081  metageneration=1
      4277  2023-11-29T22:26:06Z  gs://asap-raw-data-team-lee/metadata/v2/SUBJECT.csv#1701296766288192  metageneration=1
TOTAL: 6 objects, 59852 bytes (58.45 KiB)


In [47]:
!gsutil -u dnastack-asap-parkinsons rm -r "gs://asap-raw-data-team-lee/metadata/v2/v2_20231128"

Removing gs://asap-raw-data-team-lee/metadata/v2/v2_20231128/CLINPATH.csv#1701213902124849...
Removing gs://asap-raw-data-team-lee/metadata/v2/v2_20231128/DATA.csv#1701213902800745...
Removing gs://asap-raw-data-team-lee/metadata/v2/v2_20231128/PROTOCOL.csv#1701213903243233...
Removing gs://asap-raw-data-team-lee/metadata/v2/v2_20231128/SAMPLE.csv#1701213902582568...
/ [4 objects]                                                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m rm ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Removing gs://asap-raw-data-team-lee/metadata/v2/v2_20231128/STUDY.csv#1701213903050409...
Removing gs://asap-raw-data-team-lee/metadata/v2/v2_20231128/SUBJECT.csv#1701213902358354...
/ [6 objects]                                                                   
Operation completed over 6 objec

In [ ]:
Path.cwd()

PosixPath('/Users/ergonyc/Projects/ASAP/meta-clean')

In [133]:

!gsutil -u dnastack-asap-parkinsons cp -r "./clean/team-Lee/v2_20231130/*.csv"  "gs://asap-raw-data-team-lee/metadata/v2"

Copying file://./clean/team-Lee/v2_20231130/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Lee/v2_20231130/SUBJECT.csv [Content-Type=text/csv]...
Copying file://./clean/team-Lee/v2_20231130/SAMPLE.csv [Content-Type=text/csv]...
Copying file://./clean/team-Lee/v2_20231130/DATA.csv [Content-Type=text/csv]... 
- [4 files][ 56.5 KiB/ 56.5 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Lee/v2_20231130/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Lee/v2_20231130/PROTOCOL.csv [Content-Type=text/csv]...
\ [6 files][ 58.4 KiB/ 58.4 KiB]                                                
Operation completed over 6 objects/58.4 KiB.                                     


In [44]:
!gsutil -u dnastack-asap-parkinsons cp -r "./clean/team-Lee/*.csv"  "gs://asap-raw-data-team-lee/metadata/clean/"

Copying file://./clean/team-Lee/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Lee/SUBJECT.csv [Content-Type=text/csv]...          
Copying file://./clean/team-Lee/auxiluary_metadata.csv [Content-Type=text/csv]...
Copying file://./clean/team-Lee/SAMPLE.csv [Content-Type=text/csv]...           
\ [4 files][187.4 KiB/187.4 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Lee/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Lee/auxilarry_metadata.csv [Content-Type=text/csv]...
Copying file://./clean/team-Lee/PROTOCOL.csv [Content-Type=text/csv]...         
Copying file://./clean/team-Lee/CDE.csv [Content-Type=text/csv]...              
| [8 files][328.0 KiB/328.0 KiB]      

In [49]:
!gsutil -u dnastack-asap-parkinsons ls -al "gs://asap-raw-data-team-lee/metadata/clean"


     37741  2023-11-28T21:58:33Z  gs://asap-raw-data-team-lee/metadata/clean/CDE.csv#1701208713502213  metageneration=1
     24241  2023-11-28T21:58:31Z  gs://asap-raw-data-team-lee/metadata/clean/CLINPATH.csv#1701208711158586  metageneration=1
       968  2023-11-28T21:58:33Z  gs://asap-raw-data-team-lee/metadata/clean/PROTOCOL.csv#1701208713057031  metageneration=1
     60412  2023-11-28T21:58:32Z  gs://asap-raw-data-team-lee/metadata/clean/SAMPLE.csv#1701208712230186  metageneration=1
       969  2023-11-28T21:58:32Z  gs://asap-raw-data-team-lee/metadata/clean/STUDY.csv#1701208712490381  metageneration=1
      3078  2023-11-28T21:58:31Z  gs://asap-raw-data-team-lee/metadata/clean/SUBJECT.csv#1701208711431682  metageneration=1
    104336  2023-11-28T21:58:32Z  gs://asap-raw-data-team-lee/metadata/clean/auxilarry_metadata.csv#1701208712852367  metageneration=1
    109904  2023-09-22T10:45:50Z  gs://asap-raw-data-team-lee/metadata/clean/auxiliary_metadata.csv#1695379550068707  metagene

### copy to workflow-dev bucket

First copy each set of metadata locally ...

In [135]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/wf-credentials.json

Activated service account credentials for: [admin-workflow-dev@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [51]:
# !gsutil  ls -al "gs://asap-workflow-dev/CDE"

!gsutil  ls -al "gs://asap-workflow-dev/metadata/v2/lee"


      6243  2023-11-28T22:21:42Z  gs://asap-workflow-dev/metadata/v2/lee/CLINPATH.csv#1701210102669978  metageneration=1
     24218  2023-11-28T22:21:43Z  gs://asap-workflow-dev/metadata/v2/lee/DATA.csv#1701210103715354  metageneration=1
       968  2023-11-28T22:21:44Z  gs://asap-workflow-dev/metadata/v2/lee/PROTOCOL.csv#1701210104255218  metageneration=1
     23092  2023-11-28T22:21:43Z  gs://asap-workflow-dev/metadata/v2/lee/SAMPLE.csv#1701210103292269  metageneration=1
      1054  2023-11-28T22:21:43Z  gs://asap-workflow-dev/metadata/v2/lee/STUDY.csv#1701210103992608  metageneration=1
      4277  2023-11-28T22:21:43Z  gs://asap-workflow-dev/metadata/v2/lee/SUBJECT.csv#1701210103010639  metageneration=1
                                 gs://asap-workflow-dev/metadata/v2/lee/v2_20231128/
TOTAL: 6 objects, 59852 bytes (58.45 KiB)


In [136]:
!gsutil  cp -r "./clean/team-Lee/v2_20231130/*.csv" "gs://asap-workflow-dev/metadata/v2/lee"


Copying file://./clean/team-Lee/v2_20231130/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Lee/v2_20231130/SUBJECT.csv [Content-Type=text/csv]...
Copying file://./clean/team-Lee/v2_20231130/SAMPLE.csv [Content-Type=text/csv]...
Copying file://./clean/team-Lee/v2_20231130/DATA.csv [Content-Type=text/csv]... 
- [4 files][ 56.5 KiB/ 56.5 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Lee/v2_20231130/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Lee/v2_20231130/PROTOCOL.csv [Content-Type=text/csv]...
\ [6 files][ 58.4 KiB/ 58.4 KiB]                                                
Operation completed over 6 objects/58.4 KiB.                                     


In [56]:
!gsutil ls -al "gs://asap-workflow-dev/metadata/v2/lee"


      6243  2023-11-28T22:21:42Z  gs://asap-workflow-dev/metadata/v2/lee/CLINPATH.csv#1701210102669978  metageneration=1
     24218  2023-11-28T22:21:43Z  gs://asap-workflow-dev/metadata/v2/lee/DATA.csv#1701210103715354  metageneration=1
       968  2023-11-28T22:21:44Z  gs://asap-workflow-dev/metadata/v2/lee/PROTOCOL.csv#1701210104255218  metageneration=1
     23092  2023-11-28T22:21:43Z  gs://asap-workflow-dev/metadata/v2/lee/SAMPLE.csv#1701210103292269  metageneration=1
      1054  2023-11-28T22:21:43Z  gs://asap-workflow-dev/metadata/v2/lee/STUDY.csv#1701210103992608  metageneration=1
      4277  2023-11-28T22:21:43Z  gs://asap-workflow-dev/metadata/v2/lee/SUBJECT.csv#1701210103010639  metageneration=1
TOTAL: 6 objects, 59852 bytes (58.45 KiB)


## check file md5s

In [39]:
from utils.checksums import extract_md5_from_details, extract_md5_from_details2


In [36]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/lee-credentials.json  



Activated service account credentials for: [raw-admin-lee@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [37]:

# !gcloud storage hash "gs://asap-raw-data-team-lee/**/*.gz"  --skip-crc32c --hex  --billing-project dnastack-asap-parkinsons > hardy_hexhash.log

!gsutil -u dnastack-asap-parkinsons hash -h "gs://asap-raw-data-team-lee/**/*.gz" > lee_hexhash.log


using the module's C extension, so checksumming will run very slowly. For help
installing the extension, please see "gsutil help crcmod".



In [40]:
bucket_files_md5 = extract_md5_from_details2("lee_hexhash.log")



checksum = DATAv2[['file_name','file_MD5']]
checksum['check1'] = checksum['file_MD5'].str.strip()
checksum['check2'] = checksum['file_name'].map(bucket_files_md5)



/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_56157/1302936753.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check'] = checksum['file_name'].map(bucket_files_md5)


In [41]:
checksum[checksum.check1 != checksum.check2].file_name.to_list()
#empty means success!!


[]

In [42]:
checksum

,file_name,file_MD5,check
0,MFGHC1225_S9_L001_R1_001.fastq.gz,9977258e598d6a52130c29c71aef6925,9977258e598d6a52130c29c71aef6925
1,MFGHC1225_S9_L001_R2_001.fastq.gz,fe2cf93257801227b7072a4fb7d18792,fe2cf93257801227b7072a4fb7d18792
2,MFGHC0602_S2_L001_R1_001.fastq.gz,110ca4864cf6938faca67567bebfb6cc,110ca4864cf6938faca67567bebfb6cc
3,MFGHC0602_S2_L001_R2_001.fastq.gz,0dcc67217e43ab53bae0d0676f9bfe8b,0dcc67217e43ab53bae0d0676f9bfe8b
4,MFGPD0009_S3_L001_R1_001.fastq.gz,a2608d0bd192333b0076d7091c1c50ea,a2608d0bd192333b0076d7091c1c50ea
...,...,...,...
145,SN_1973_PD_S1_L000_R2_001.fastq.gz,53f6c6b4a00299fb41f32b34509835fa,53f6c6b4a00299fb41f32b34509835fa
146,SN_2005_PD_S1_L001_R1_001.fastq.gz,5bde8fdd8ab28e2c00e3ae327fecc80b,5bde8fdd8ab28e2c00e3ae327fecc80b
147,SN_2005_PD_S1_L001_R2_001.fastq.gz,14364dc52760fbd3ad08b5ca582d849b,14364dc52760fbd3ad08b5ca582d849b
148,SN_2038_PD_S1_L000_R1_001.fastq.gz,3215d4e4a68a85546183e820b74cac1f,3215d4e4a68a85546183e820b74cac1f
